In [5]:
import pandas as pd
df1 = pd.read_csv("../sheet1_1.csv")
df0 = pd.read_csv("../sheet0_num.csv")

df = pd.merge(df0, df1, on='id').fillna(0).drop(
    ['id', 'ornament', 'color', 'degree', 'weathered'], axis=1)
df.to_csv('./q4_data.csv')
df1 = df[df['type'] == 1]
df0 = df[df['type'] == 0]


In [7]:
import numpy as np
np1 = np.array(df1)
np0 = np.array(df0)


array([[1.000e+00, 6.933e+01, 0.000e+00, 9.990e+00, 6.320e+00, 8.700e-01,
        3.930e+00, 1.740e+00, 3.870e+00, 0.000e+00, 0.000e+00, 1.170e+00,
        0.000e+00, 0.000e+00, 3.900e-01],
       [1.000e+00, 8.705e+01, 0.000e+00, 5.190e+00, 2.010e+00, 0.000e+00,
        4.060e+00, 0.000e+00, 7.800e-01, 2.500e-01, 0.000e+00, 6.600e-01,
        0.000e+00, 0.000e+00, 0.000e+00],
       [1.000e+00, 6.171e+01, 0.000e+00, 1.237e+01, 5.870e+00, 1.110e+00,
        5.500e+00, 2.160e+00, 5.090e+00, 1.410e+00, 2.860e+00, 7.000e-01,
        1.000e-01, 0.000e+00, 0.000e+00],
       [1.000e+00, 6.588e+01, 0.000e+00, 9.670e+00, 7.120e+00, 1.560e+00,
        6.440e+00, 2.060e+00, 2.180e+00, 0.000e+00, 0.000e+00, 7.900e-01,
        0.000e+00, 0.000e+00, 3.600e-01],
       [1.000e+00, 6.158e+01, 0.000e+00, 1.095e+01, 7.350e+00, 1.770e+00,
        7.500e+00, 2.620e+00, 3.270e+00, 0.000e+00, 0.000e+00, 9.400e-01,
        6.000e-02, 0.000e+00, 4.700e-01],
       [1.000e+00, 6.765e+01, 0.000e+00, 7.370e+00

In [15]:
import pandas as pd
import numpy as np

def loadDataSet(df):
    return np.array(df)


def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
    C1.sort()
    return list(map(frozenset, C1))


def scanD(D, CK, minSupport):
    ssCnt = {}
    for tid in D:
        for can in CK:
            if can.issubset(tid):
                if not can in ssCnt:
                    ssCnt[can] = 1
                else:
                    ssCnt[can] += 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]/numItems
        if support >= minSupport:
            retList.insert(0, key)
        supportData[key] = support
    return retList, supportData

# 频繁项集两两组合


def aprioriGen(Lk, k):
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk):
            L1 = list(Lk[i])[:k-2]
            L2 = list(Lk[j])[:k-2]
            L1.sort()
            L2.sort()
            if L1 == L2:
                retList.append(Lk[i] | Lk[j])
    return retList


def apriori(dataSet, minSupport=0.3):
    C1 = createC1(dataSet)
    D = list(map(set, dataSet))
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while(len(L[k-2]) > 0):
        CK = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, CK, minSupport)
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

# 规则计算的主函数


def generateRules(L, supportData, minConf=0.7):
    bigRuleList = []
    for i in range(1, len(L)):
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if(i > 1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList


def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    prunedH = []
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq]
        if conf >= minConf:
            print(freqSet-conseq, '--->', conseq, 'conf:', conf)
            brl.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH


def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    m = len(H[0])
    if (len(freqSet) > (m+1)):
        Hmp1 = aprioriGen(H, m+1)
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        if(len(Hmp1) > 1):
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)


if __name__ == '__main__':

    df1 = pd.read_csv("../sheet1_1.csv")
    df0 = pd.read_csv("../sheet0_num.csv")

    df = pd.merge(df0, df1, on='id').fillna(0).drop(
    ['id', 'ornament', 'color', 'degree', 'weathered'], axis=1)
    #df.to_csv('./q4_data.csv')
    df1 = df[df['type'] == 1]
    df0 = df[df['type'] == 0]
    dataSet = loadDataSet(df0)
    L, supportData = apriori(dataSet)
    rules = generateRules(L, supportData, minConf=0.3)


In [29]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

df1 = pd.read_csv("../sheet1_1.csv")
df0 = pd.read_csv("../sheet0_num.csv")

df = pd.merge(df0, df1, on='id').drop(
    ['id', 'ornament', 'color', 'degree', 'weathered'], axis=1)

df1 = df[df['type'] == 1].drop(['type'],axis=1).applymap(
    lambda x: 1 if str(x) != 'nan' else 0)
df0 = df[df['type'] == 0].drop(['type'], axis=1).applymap(
    lambda x: 1 if str(x) != 'nan' else 0)
frequent_itemsets = apriori(
    df0, min_support=0.50, use_colnames=True)
print(frequent_itemsets[:10])


    support itemsets
0  1.000000    (sio)
1  0.918367    (cao)
2  0.612245    (mgo)
3  1.000000    (alo)
4  0.551020    (feo)
5  0.918367    (cuo)
6  1.000000    (pbo)
7  0.938776    (bao)
8  0.836735     (po)
9  0.816327    (sro)


/home/cqy/anaconda3/envs/tinybert/lib/python3.9/site-packages/mlxtend/frequent_patterns/fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [49]:
from mlxtend.frequent_patterns import association_rules
 
association_rule = association_rules(frequent_itemsets,metric='confidence',min_threshold=0.9)
rules[(rules['lift'] > 1) | (rules['lift'] < 1)][:50].to_csv('rules.csv')
